In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = save['train_dataset']
    train_labels = save['train_labels']
    valid_dataset = save['valid_dataset']
    valid_labels = save['valid_labels']
    test_dataset = save['test_dataset']
    test_labels = save['test_labels']
    del save
    print('Training Set', train_dataset.shape, train_labels.shape)
    print('Validation Set', valid_dataset.shape, valid_labels.shape)
    print('Test Set', test_dataset.shape, test_labels.shape)

Training Set (200000, 28, 28) (200000,)
Validation Set (10000, 28, 28) (10000,)
Test Set (10000, 28, 28) (10000,)


In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
    dataset = dataset.reshape((-1, image_size*image_size)).astype(np.float32)
    labels = (np.arange(num_labels) == labels[:, None]).astype(np.float32)
    return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

print('Training Set', train_dataset.shape, train_labels.shape)
print('Validation Set', valid_dataset.shape, valid_labels.shape)
print('Test Set', test_dataset.shape, test_labels.shape)

Training Set (200000, 784) (200000, 10)
Validation Set (10000, 784) (10000, 10)
Test Set (10000, 784) (10000, 10)


In [4]:
#train_subset = 10000
beta = 0.01

graph = tf.Graph()
with graph.as_default():
    # Input Data
    # Load Training, Validation and Test data into Constants attached to graph
    #tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    #tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_train_dataset = tf.constant(train_dataset)
    tf_train_labels = tf.constant(train_labels)
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    # These are parameters that we are going to train
    # Weight matrix is randomly initialized 
    # Biases initialized to zero
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training Computation
    # Linear -- Multiply Inputs with Weights and add Biases
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    # Compute softmax and cross-entropy loss and take average
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
    # L2 Regularization
    regularize = tf.nn.l2_loss(weights)
    loss = tf.reduce_mean(loss + beta * regularize)
    
    # Optimizer - Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.5).minimize(loss)
    
    # Predictions
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
num_steps = 801
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
            / predictions.shape[0])


with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        # Run the computations
        _, l, predictions = sess.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, train_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 34.638893
Training accuracy: 90.7%
Validation accuracy: 87.6%
Loss at step 100: 11.064051
Training accuracy: 99.5%
Validation accuracy: 96.4%


In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate Minibatch
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

In [ ]:
num_nodes = 1024
batch_size = 128
beta = 0.01


graph = tf.Graph()

with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    w1 = tf.Variable(tf.truncated_normal([image_size*image_size, num_nodes]))
    b1 = tf.Variable(tf.zeros([num_nodes]))

    w2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    b2 = tf.Variable(tf.zeros([num_labels]))
    
    
    # Training & Dropout
    z1 = tf.matmul(tf_train_dataset, w1) + b1
    a1 = tf.nn.relu(z1)
    keep_prob = tf.placeholder(tf.float32)
    z2 = tf.nn.dropout(a1, keep_prob)
    z2 = tf.matmul(a1, w2) + b2
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=z2))
    
    ## L2 Regularization
    regularize = tf.nn.l2_loss(w1) + tf.nn.l2_loss(w2)
    loss = tf.reduce_mean(loss + beta * regularize)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(z2)
    
    
    ## Validation Prediction
    z1 = tf.matmul(tf_valid_dataset, w1) + b1
    a1 = tf.nn.relu(z1)
    z2 = tf.matmul(a1, w2) + b2
    valid_prediction = tf.nn.softmax(z2)
    
    ## Test Prediction
    z1 = tf.matmul(tf_test_dataset, w1) + b1
    a1 = tf.nn.relu(z1)
    z2 = tf.matmul(a1, w2) + b2
    
    test_prediction = tf.nn.softmax(z2)

In [ ]:
num_steps = 3001

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate Minibatch
        batch_data = train_dataset[offset:(offset+batch_size), :]
        batch_labels = train_labels[offset:(offset+batch_size), :]
        
        feed_dict = {tf_train_dataset: batch_data, tf_train_labels: batch_labels}
        _, l, predictions = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step%500 == 0):
            print("Minibatch loss at step %d: %f" % (step, l))
            print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
            print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))